In [ ]:
# Montar o google drive no coolab
from google.colab import drive
drive.mount('/content/drive')

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from transformers import BertTokenizer, RobertaTokenizer, BertTokenizerFast, BertModel, BertForSequenceClassification
import torch
import torch.nn.functional as F
import torch.nn as nn

from sklearn.metrics import classification_report
from sklearn import metrics

import pickle
import re
import os
from tqdm import tqdm
import ast
import random
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset


## Importando DataFrames

In [ ]:
# df_train_use.to_csv('clean_train_corpus.csv', index=False)
# df_train_use = pd.read_csv('C:\\Users\\Bruno Scale\\Documents\\Mestrado\\Experimentos\\data\\clean\\clean_train_corpus.csv')
df_train_use = pd.read_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/depression/BERT/clean_train_corpus.csv')

# df_test_use.to_csv('clean_test_corpus.csv', index=False)
# df_test_use = pd.read_csv('C:\\Users\\Bruno Scale\\Documents\\Mestrado\\Experimentos\\data\\clean\\clean_test_corpus.csv')
df_test_use = pd.read_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/depression/BERT/clean_test_corpus.csv')

In [ ]:
df_test_use.head()

## Funções para métrica de avaliação f1-latency


Para cálculo da métrica de interesse f1-*latency* são necessárias algumas etapas representadas pelas funções abaixo.

In [ ]:
def value_p(k):
    """Get the penalty value for the F latency measure.

    Parameters
    ----------
    k : int
        Median number of posts from the positive users.

    Returns
    -------
    penalty : float
        Penalty to use.
    """
    return -(np.log(1 / 3) / (k - 1))


def f_penalty(k, _p):
    """Get the penalty of the current user delay.

    Parameters
    ----------
    k : int
        Current user delay.
    _p : float
        Penalty.

    Returns
    -------
    f_penalty : float
        Penalty latency.
    """
    return -1 + (2 / (1 + np.exp((-_p) * (k - 1))))


def speed(y_pred, y_true, d, p):
    """Get speed for every user correctly classified as positive."""
    penalty_list = [
        f_penalty(k=d[i], _p=p)
        for i in range(len(y_pred))
        if y_pred[i] == 1 and y_true[i] == 1
    ]

    if len(penalty_list) != 0:
        return 1 - np.median(penalty_list)
    else:
        return 0.0


def f_latency(labels, true_labels, delays, penalty):
    """F latency metric.

    Metric proposed by Sadeque and others in [1]_.

    Parameters
    ----------
    labels : list of int
        Predicted label for each user.
    true_labels : list of int
        True label for each user.
    delays : list of int
        Decision delay for each user.
    penalty : float
        Penalty. Defines how quickly the penalty should increase.

    Returns
    -------
    f_latency_metric : float
        F latency measure.

    References
    ----------
    .. [1] `Sadeque, F., Xu, D., & Bethard, S. (2018, February). Measuring the
        latency of depression detection in social media. In Proceedings of the
        Eleventh ACM International Conference on Web Search and Data Mining
        (pp. 495-503).`_
    """
    f1_score = metrics.f1_score(y_pred=labels, y_true=true_labels, average="binary")
    speed_value = speed(y_pred=labels, y_true=true_labels, d=delays, p=penalty)

    return f1_score * speed_value

## Data prep

In [ ]:
df_train_use_BERT = df_train_use.copy()
df_test_use_BERT = df_test_use.copy()

In [ ]:
# Transformo a timeline em listas
def timeline_list(text):
    return text.split('$END_OF_POST$')

df_test_use_BERT['List_CleanText'] = df_test_use_BERT['CleanText'].apply(timeline_list)


# Faço a contagem de posts
def contar_posts(texto):
    return texto.count('$END_OF_POST$') + 1

df_test_use_BERT['Qt_Messages'] = df_test_use_BERT['CleanText'].apply(contar_posts)

Vou utilizar a estratégia de manter apenas a quantidade de tweets da menor timeline de todas do córpus.

In [ ]:
menor_timeline = df_test_use_BERT['Qt_Messages'].min()

def min_timeline(lista):
    return lista[- menor_timeline:]

df_test_use_BERT['List_CleanText_min'] = df_test_use_BERT['List_CleanText'].apply(min_timeline)

## Fine-tunning BERT

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Definindo o modelo BERT e tokenizer
# model_name = "neuralmind/bert-base-portuguese-cased"
model_name = 'pablocosta/bertabaporu-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Definindo o dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['Text']
        label = self.dataframe.iloc[idx]['Diagnosed_YN']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = CustomDataset(df_train_use_BERT, tokenizer)
# test_dataset = CustomDataset(test_df, tokenizer)

# Definindo o DataLoader
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Definindo os parâmetros de treinamento
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3

# Treinamento do modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Average training loss: {avg_train_loss}')

# Avaliando o modelo
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(train_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculando a acurácia
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
print(f'Accuracy: {accuracy}')


In [ ]:
print(classification_report(true_labels, predictions))

In [ ]:
# import pickle

# # Salvando o estado do modelo
# torch.save(model.state_dict(), 'modelo_final.pth')

# # Salvando outros metadados importantes, se necessário
# # Por exemplo, salvar o tokenizer
# with open('bert.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f)

model_name = 'pablocosta/bertabaporu-base-uncased'
# Para o modelo já treinado, quero carregar
# Carregar o modelo a partir do arquivo .pkl, se necessário
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Carregando o estado do modelo
model.load_state_dict(torch.load('modelo_final.pth'))

# Carregando o tokenizer, se necessário
with open('bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


## Política DMC - Momento de Decisão da Classificação
Os parâmetros usados nesta política foram de limiar = 0.5 e número mínimo de mensagens lidas de 20.

### Utilizando a política como está descrita no artigo

A política foi inspirada no trabalho descrito em Loyola et al. (2022) com as camadas de Decisão do Momento de Classificação (DMC) e Classificação Parcial da Informação (CPI) com adaptações necessárias para o português no córpus SetembroBR.

In [ ]:
# parâmetros
delta = 0.5
n = 20

In [ ]:
import torch
import pickle
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm

# Colocar o modelo em modo de avaliação
model.eval()

# Criar lista para armazenar inputs tokenizados
tokenized_inputs = []

print('tokenizando todos os dados..')
# Tokenizar todos os dados
for text in tqdm(df_test_use_BERT['Text']):
    tokenized_input = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    tokenized_inputs.append(tokenized_input)

In [ ]:
# Realizar a inferência
qtd_msgs_lidas = []
df_test_use_BERT['prediction'] = 0

print('aplicando a política..')
for i, tokenized_input in enumerate(tqdm(tokenized_inputs)):
    for j in range(len(df_test_use_BERT['List_CleanText_min'][i])):
        inputs_temp = {k: tokenized_input[k][:, j:j+10] for k in tokenized_input}

        # Realizar a inferência
        with torch.no_grad():
            outputs = model(**inputs_temp)

        # Interpretar os resultados
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        prob = torch.softmax(logits, dim=1)

        # Converter as previsões de volta para labels
        predicted_labels = [model.config.id2label[p.item()] for p in predictions]
        y_pred = [model.config.id2label[p.item()] for p in predictions]

        if int(y_pred[0][-1]) == 1 and prob.max() >= delta:
            df_test_use_BERT.loc[i, 'prediction'] = 1
            break
    qtd_msgs_lidas.append(j + n)

df_test_use_BERT['qtd_msgs_lidas'] = qtd_msgs_lidas

In [ ]:
df_test_use_BERT.to_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/BERT/prediction_BERT_MinTL.csv')

In [ ]:
df_train_use_bert_teste_1 = df_train_use_BERT.copy()

# parâmetros
delta = 0.5
n = 20

# Criando objetos auxiliares
qtd_msgs_lidas = []
df_train_use_bert_teste_1['prediction'] = 0

model.eval()

# Avaliando a previsão e calculando a quantidade de mensagens lidas
# for i in tqdm(range(5)): # Caso de teste
for i in tqdm(range(len(df_train_use_bert_teste_1))):
  for j in range(len(df_train_use_bert_teste_1['List_CleanText'][i])):
    lista_temp = ['$END_OF_POST$'.join(df_train_use_bert_teste_1['List_CleanText'][i][j:j+n])]


    # Tokenizar os novos dados
    inputs = tokenizer(lista_temp, padding=True, truncation=True, return_tensors="pt")

    # Realizar a inferência
    with torch.no_grad():
      outputs = model(**inputs)

    # Interpretar os resultados
    logits = outputs.logits
    prob = torch.softmax(logits, dim=1)
    predicted_labels = torch.argmax(logits, dim=1)
    y_pred = [model.config.id2label[p.item()] for p in predicted_labels]

    if y_pred == 1 and prob.max() >= delta:
      # print(f'Quantidade de mensagens lidas: {j+n}')
      df_train_use_bert_teste_1['prediction'][i] = 1

      break
  qtd_msgs_lidas.append(j+n)

df_train_use_bert_teste_1['qtd_msgs_lidas'] = qtd_msgs_lidas
df_train_use_bert_teste_1.head()


## Avaliação

In [ ]:
df_test_use_BERT = pd.read_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/BERT/prediction_BERT_MinTL.csv')
df_test_use_BERT.head()

In [ ]:
y_train_teste_1 = df_test_use_BERT['Diagnosed_YN']
y_pred_teste_1 = df_test_use_BERT['prediction']
print(classification_report(y_train_teste_1, y_pred_teste_1))

In [ ]:
f1_latency = f_latency(df_test_use_BERT['prediction'], df_test_use_BERT['Diagnosed_YN'], df_test_use_BERT['qtd_msgs_lidas'], 0.0078)
f1_latency

#### Tempo de antecipação

In [ ]:
df_test_use_BERT['List_Timeline'] = df_test_use_BERT['List_Timeline'].apply(eval)

In [ ]:
from datetime import datetime

# Função para converter a lista de datas em objetos datetime
def converter_datas(lista_datas_str):
    # Remover os colchetes e espaços em branco
    # lista_datas_str = lista_datas_str.strip('[]').replace("'", "").split(', ')
    # Converter cada data para datetime
    return [pd.to_datetime(data_str, format='%d-%b-%Y %H:%M') for data_str in lista_datas_str]


# Função para calcular a diferença em dias entre duas datas
def calcular_diferenca_dias(data1, data2):
    delta = data1 - data2
    return delta.days

# Aplicando a conversão de datas ao DataFrame
df_test_use_BERT['List_Timeline'] = df_test_use_BERT['List_Timeline'].apply(converter_datas)

# Aplicando a função ao DataFrame
df_test_use_BERT['ultima_data_tweet_lido'] = df_test_use_BERT.apply(lambda row: row['List_Timeline'][row['qtd_msgs_lidas'] - 1], axis=1)
df_test_use_BERT['ultima_data_tweet_lista'] = df_test_use_BERT.apply(lambda row: row['List_Timeline'][-1], axis=1)
df_test_use_BERT['dias_antecipados'] = df_test_use_BERT.apply(lambda row: calcular_diferenca_dias(row['ultima_data_tweet_lista'], row['ultima_data_tweet_lido']), axis=1)

df_test_use_BERT.head()

In [ ]:
dias_antecipacao_geral = df_test_use_BERT['dias_antecipados'].mean()
dias_antecipacao_pos = df_test_use_BERT[df_test_use_BERT['Diagnosed_YN']==1]['dias_antecipados'].mean()
print(f'Média de dias de antecipação de risco geral: {dias_antecipacao_geral}')
print(f'Média de dias de antecipação de risco da classe positiva: {dias_antecipacao_pos}')